<a href="https://colab.research.google.com/github/falconlee236/DeepLearningFrom_Scratch/blob/main/ch04/Chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4 Neural network training

**4.2 loss function**

*4.2.1 sum of squares for error, SSE*

In [ ]:
def sum_squares_error(y, t):
  return 0.5 * np.sum((y - t) ** 2)

In [ ]:
import numpy as np

# answer is 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# example1 : estimate probablity of number 2 to most highest probablity(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

In [ ]:
# example2 : estimate probablity of number 7 to most highest probablity(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

the smaller value of loss function is, the better answer has

*4.2.2 cross entropy error, CEE*

In [ ]:
def cross_entropy_error(y, t):
  delta = 1e-7 # log0 = -inf/ protect -inf
  return -np.sum(t * np.log(y + delta))

In [ ]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

In [ ]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))